# Task 3 - The Prototype NLP task

### The ability to search for items has increased exponentially due to the internet. A recommendation engine is a way of reducing the inherent difficulty such searches provide by giving a more focussed output based on specific inputs, rather than having to wade through the multitude of search reults.

### The system built for this project is a prototype to show that the task was possible. The user can choose to input a set of criteria for the drivers that they are searching for, including the brand, handedness, price, loft, shaft type and flex and any other keywords relevant to their search. This user input is requested and then used to provide a recommendation of the top ten matches for a golf driver. It is an unsupervised task as there are no user ratings provided with which to determine the accuracy of the recommendations.

### The system can be improved in many ways by adding more websites from which options for drivers are increased and also getting more reviews. The data input method can also be refined to focus the users requirements. For example searching on the features selected (brand/model/dexterity/loft/shaft) to reduce the number of options before using the review and verdict fields to look for specific keyword matches. In fact the addition of the review and verdict fields seems to have actually made the recommender worse as there are too many words to compare with and the reviews all contain similar wording. Fixing this will be the focus of future work.

## *Brief Literature Review*

### Due to the nature of the user inputs the system was built using the FuzzyWuzzy python string matching function. This utilised the ability of function to the take any number of input values and compare them against the known values stored. This literature review will look at several other recommendation systems that use this technique and discuss the relative merits of this methodology. No other specifc golf driver recommendation systems were found, however, such systems exist on most golf retailer sites already where suggestions are made based on what other people have bought for example.

### In the discussion of their implementation of a movie recommender system Manis (2020) discuss the advantages of using the fuzzywuzzy by allowing for the mistyping of input, the string match is still able to find similar movies. They use the example of typing 'avengrs' for a recommendation for the movie 'Avengers'.  

### A global citation recommendation system was developed by Ayala-Gomez (2018) in which they discuss using the fuzzywuzzy package to merge 2 datasets based on the titles of the papers.  Their task is similar to this one in that they aimed 'to find papers that are relevant to a short description (abstract) of a new paper, provided as input.' The advantages of using the fuzzywuzzy package for this task are stated as allowing for different string matching.

### The advantages of using the fuzzywuzzy package are also discussed by Dobin(2020) Given that the data dictionary was not clean, often times there was data match errors between our user data and the dictionary while meant that we had to implement fuzzy matching for the titles.

### Based on these few reviews the implentation of a recommender system using the fuzzywuzzy package was a appropriate methodology for this task.


## Rationale for selection of the NLP task

### Nowadays almost any retail website you visit will provide you with a selection or recommendation of what products or services similar users purchased or liked - think Amazon or Ebay for example. 

### The rationale behind this project was to attempt to do something similar but not on such a large scale. Essentially a person who is interested in buying a new golf driver for themselves can be bombarded with information. Depending on their level of knowledge they may need some help when deciding which one to buy. This is the role this recommender system will fill. Data was scraped from a retail website of golf driver listings and of reviews from a golf magazine site. These two sets of data were then joined for use within the recommendation engine. The advantages to the implementation of this system using the fuzzywuzzy package was discussed above.

## Data pre-processing

### As mentioned above there were two dataframes saved as csv files from Task 2. The first requirement was to now join those two dataframes based on the Driver_Name field stored in both. It was hoped that with 220 reviews that almost all of the 98 drivers for sale would have a matching review. This matching was done using the fuzzy_panda package using the Levenshtein distance method to calculate similarities, in this case between the similarities of the driver name in both dataframes. A visual inspection is provided to show the accuracy in joining driver with review. It was not perfect but acceptable.

### The inputs were simply entered as a string. This is where I believe more work could be done to get the data entry side more robust. A web page with drop down menus and maybe a text box for entering a description would best suit this system. With the outputs displayed as a table with the name, link to buy, price, dexterity, flex, condition and sim_score given.

### The natural language processing toolkit was used to tokenize, lemmatize and stem both the inputs and the reviews as well as removing the stop words. 

## *Performance*

### Assessing the performance of this recommendation system was not able to be performed using the usual traing/test set methods. A simple explanation of the search results given is provided in the results section where several test runs were done and the output examined. Overall there needs to be some improvement with regards to the data entry method that would allow for better filtering of the choices (i.e. dexterity/flex type/condition etc.) before using the review and verdict fields for keywords.

### To test the performance of the recommender based solely on the specifications and not the review or verdict I ran a comparison by implementing a recommender without them. These results proved much closer to the user input values if specifications were entered. If however the query does not include the specification but is something like 'best driver toe player with high handicap' then include the review in the recommender does actually add a lot of value. 

### So there is a question that needs to be asked here. Who will the end users be. Will they be knowledgeable people or those just learning ?? Perhaps there could be two systems one for each type of user.

### *References*

#### Sharma, M., Kashyap, A., & Saini, H. (2020). Movies Recommendation System.

#### Ayala-Gómez F, Daróczy B , Benczúr A , Mathioudakis M & Gionis A (2018) Global citation recommendation using knowledge graphs. Journal of Intelligent & Fuzzy Systems 34 (2018) 3089–3100 DOI:10.3233/JIFS-169493

#### Dobkin, B., & Satheesh, B. (2020). To Compliment a Complement? Modeling New Avenues for Podcast Content Discovery.

## *The code for the recommender is provided below.*

In [18]:
import re
import pandas as pd
import fuzzy_pandas as fpd
import numpy as np

# Libraries for nltk
import nltk   #natural language toolkit
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#fuzzywuzzy for recommender
from fuzzywuzzy import fuzz

# to output nice dataframe table
from IPython.display import display 

# dataframe display options
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

pd.set_option('mode.chained_assignment', None) # to silence the SettingWithCopy warning

In [4]:
# load the df_drivers and df_golfmonthly_reviews dataframe s
df_drivers   = pd.read_csv('df_drivers.csv')
df_gm_reviews = pd.read_csv('df_golfmonthly_reviews.csv')


In [5]:
# Lemmatization and stemming process
'''
Words in the third person are changed to first person and verbs in past and future tenses are changed into the present by the 
lemmatization process. Words are reduced to their stems by the PorterStemmer
'''

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
def tokenize_and_lem_stem(text):
    # tokenization to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    
    for t in tokens:
        if t not in stopwords.words():
            if re.search('[a-zA-Z0-9]', t): # use regex to get only letters and numbers - include capitals just in case
                filtered_tokens.append(ps.stem(t)) 
        lem = [lemmatizer.lemmatize(ft) for ft in filtered_tokens]
    
    return lem

In [6]:
# check the data
df_drivers.head()

,Unnamed: 0,Driver_Name,Link,Price,Loft,Dexterity,Shaft,Flex,Grip,Condition
0,0,callaway mavrik,https://www.golftraders.com.au/new-callaway-mavrik-driver-10.5-graphite-with-shaf,549.0,10.5,right,graphite - options available,options available,lamkin crossline black midsize rubber grip,new
1,1,cobra cobra ladies fly-z ultra marine,https://www.golftraders.com.au/cobra-ladies-fly-z-marine-driver-10.5-13.5-ladies,329.0,10.5-13.5 adjustable,left,graphite matrix vlct -sp 50g,ladies,cobra winn+ master wrap rubber grip,new
2,2,cobra max blue offset high launch,https://www.golftraders.com.au/cobra-max-blue-offset-driver-hl-15-graphite-ladies,399.0,15,left,matrix mfs 45x4 white tie graphite,ladies,cobra winn master wrap soft polymer grip,new
3,3,cobra king radspeed,https://www.golftraders.com.au/new-cobra-king-radspeed-driver-9-motore-x-f1-stiff,799.0,9,right,graphite motore x f1,stiff,cobra connect rubber grip,new
4,4,cobra king sz black / white,https://www.golftraders.com.au/new-cobra-king-sz-white-black-driver-10.5-graphite,549.0,10.5 adjustable,right,graphite tensei av series 65,regular,cobra connect rubber grip,new


In [7]:
# drop the unnamed column
df_drivers.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
# check the data
df_gm_reviews.head(1)

,Unnamed: 0,Driver_Name,Link,Review,Verdict
0,0,tour edge exs 220,https://www.golfmonthly.com/reviews/drivers/tour-edge-exs-220-driver-review,part of the exotics line from tour edge which is the brand s more high-end club series this 460cc driver is relatively affordable compared to some of the best drivers on the market it has a stretched out look that many different players and abilities will like in a classic shape its carbon fibre crown construction replete with a visible weave and toe titanium body and beta titanium clubface means weight is positioned to maximise forgiveness collectively improving the consistency of launch and spin to boot behind the clubface are 33 thick and thin mini-trampolines that make for a hotter and faster ball flight with better mis-hit performance tour edge bills this as one of its fastest drivers ever the exs 220 driver has an adjustable 9-gram back weight to create one of the highest m.o.i ratings ever manufactured in a driver additional weights will be available in 3 grams 6 grams 11 grams and 14 grams to help dial in the preferred swing weight and trajectory on the hosel you can tune the loft up or down two degrees from the base loft what s really awesome is how it helps alter your experience on the tee box there s the solid impact sound due to a sound diffusion bar inside the clubhead a concept borrowed from concert halls to enhance the richness of sound in this case it just makes impact seem powerful without being overly loud in our testing we consistently achieved plenty of trajectory along with tight dispersion there are two stock shaft options ours had a mitsubishi fubuki hd50 which felt a little bit limp with the loft set at 10.5° ball flight seemingly played at a trajectory several degrees higher and that likely contributed to a distance that was satisfyingly competitive with anything else from other major brands this is clearly the longest and most-powerful exotics driver to hit the market yet,the ball flight on offer here is fantastic when hit out of the screws it 's arguably tour edge 's best driver to date expect a soaring tall ball flight even if you re typically a low-ball hitter at this price it has compelling value too.


# The task now is to link the reviews to the drivers, as much as possible, there are 220 reviews and 99 drivers on sale.
## To do this the Driver_Name column of each data frame needs to be matched. This is done using the fuzzy_pandas function

In [10]:
# Need to change Link and Driver_Name in review dataframe so when they get joined there aren't duplicate column headings
df_gm_reviews.rename(columns={'Driver_Name':'Driver_Name_review','Link':'Link_review'},inplace=True)

## Using the fuzzy_pandas package it is possible to perform a fuzzy_merge based on the Driver_Name columns of each dataframe. It is not perfect but allows for the next step - the recommendation engine

In [11]:
'''
A results dataframe is created on the basis of the fuzzy pandas merge using the levelshtein distance. 
'''

results = fpd.fuzzy_merge(df_drivers, df_gm_reviews,
            left_on='Driver_Name',
            right_on='Driver_Name_review',
            method='levenshtein',
            threshold=0.70)

## Below results of the fuzzy panda dataset merge are shown. Comparisons can be done between the Driver_Name field. Whilst the merge is by no means perfect there are some possible reasons for this. Firstly the names in both datasets may not have been exactly similar, for example several results show a trailing 's' in the review Driver_Name. These differences may result in diefferent brivers being selected but the provision of a recommendation of 'similar' products is a starting point for the users search.

In [12]:
results[['Driver_Name','Driver_Name_review']]

,Driver_Name,Driver_Name_review
0,callaway mavrik,callaway mavrik s
1,taylormade sim 2,taylormade sim2 s
2,taylormade sim 2,taylormade sim s
3,taylormade sim 2,taylormade m2
4,callaway great big bertha hawk eye,callaway great big bertha epic s
...,...,...
81,taylormade r1,taylormade r9
82,taylormade tour burner,taylormade aeroburner
83,taylormade tour burner,taylormade tour burner
84,titleist ts2,titleist tsi2


In [13]:
# Rename the results as a new dataframe df_combined
df_combined = results

In [14]:
# View the data - note the large spacing due to the lenght of the Link_review field.

df_combined.head(1)

,Driver_Name,Link,Price,Loft,Dexterity,Shaft,Flex,Grip,Condition,Unnamed: 0,Driver_Name_review,Link_review,Review,Verdict
0,callaway mavrik,https://www.golftraders.com.au/new-callaway-mavrik-driver-10.5-graphite-with-shaf,549.0,10.5,right,graphite - options available,options available,lamkin crossline black midsize rubber grip,new,17,callaway mavrik s,https://www.golfmonthly.com/reviews/drivers/callaway-mavrik-drivers-review,rogue was an underrated driver range and surpassing it was always going to be tricky for callaway its replacement mavrik has an updated flash face designed by a super computer you can read more about the new technology here so it promised a great deal… and delivered to a degree we tested the standard and sub zero mavrik drivers indoors on the foresight sports gcquad launch monitor in 9° loft and the new aldila rogue 60x shaft which is one of the stock options we ll be sure to add in the performance of the max driver to this review once we have tested it as it s arrival was delayed on first inspection the orange colour pops certainly catch your eye on the shelf some may say it cheapens the look but it s certainly preferable to all black at address they look modern with the carbon fibre weave pattern visible a traditional shape without too many elements to distract you the sub zero model is shorter from front to back while the standard is a little larger and the max has the most stretched out look of all three this image highlights the unusual cyclone aero shape of the standard mavrik driver the mavrik drivers feel incredibly solid like almost no energy is lost between the collision of club and ball with a powerful thud sound we really enjoyed we were expecting the cyclone shape of the standard model to give us more clubhead speed compared to the sub zero but that wasn t the case in fact we swung the mavrik sub zero driver nearly 2mph faster perhaps due to the smaller size the sub zero in its more forgiving weight setting with the heavier 14g weight at the rear performed the best for us while also being playable on the course while it didn t quite give us the ball speed of the standard model the lower spin helped us achieve carries that were four yards longer on average the standard model is unquestionably more forgiving and easier to hit straighter was faster and more efficient off the face and will benefit slower swing speeds or golfers that need more spin to maximize distance it gave us 300 rpm more spin on average out on the course we were surprised at how easy it was to control direction with the mavrik sub zero without losing out on distance distance with control is always a winning formula and this seemed to deliver it especially on slight mishits it can t save a really bad strike or swing but it s certainly more user-friendly than rogue we continually check thousands of prices to show you the best deals if you buy a product through our site we will earn a small commission from the retailer a sort of automated referral fee but our reviewers are always kept separate from this process you can read more about how we make money in our ethics policy golf monthly live prices about our deals callaway mavrik drivers in stock £339 view callaway mavrik ladies golf driver in stock £349 view callaway mavrik driver in stock £349 view callaway golf mens black mavrik stiff right hand even flow riptide 50 golf driver size 9 american golf in stock £349 view callaway mavrik drivers rh mens/ ladies regular/ stiff in stock £689 view powered by golf monthly,mavrik does n't appear to represent a big leap forward from rogue nor does it noticeably outperform epic flash but these drivers certainly made finding fairways a little easier without having to sacrifice distance and the different models in the range mean every player type is catered for.


In [373]:
# Drop the unnamed column
#df_combined.drop('Unnamed: 0',axis=1, inplace=True)

# The recommender will take a series of inputs (prompted from the user) and then use the fuzzywuzzy method to compare the inputs with the review and verdict to recommend a driver

In [42]:
'''
This function takes a series of user inputs taken from the run_recommender function and returns 
FuzzyWuzzy token_set_ratio between a list of drivers matching the inputs.
It needs to be this ratio as that ignores repated words.
'''

def recommend_driver(details):
    N = 5 #default number of results to return
    scores = []
    results = []
    length = df_combined.shape[0]
    
    for i in range(length):
            driver_details = df_combined.loc[i,'Driver_Name'] + str(df_combined.loc[i,'Driver_Name']) + str(df_combined.loc[i,'Price']) +\
                             str(df_combined.loc[i,'Loft'])  +  str(df_combined.loc[i,'Dexterity'])   + str(df_combined.loc[i,'Shaft']) +\
                             str(df_combined.loc[i,'Flex']) +str(df_combined.loc[i,'Condition'])+  str(df_combined.loc[i,'Condition'])  +\
                             str(df_combined.loc[i,'Review']) + str(df_combined.loc[i,'Verdict'])
            driver_details = tokenize_and_lem_stem(driver_details) # stored values
            requested     = tokenize_and_lem_stem(str(details))    # user input values
            scores.append(fuzz.token_set_ratio(driver_details, requested)  ) # similarity measure 
    
    # which columns to extract
    
    recommended = df_combined.iloc[:,[0,1,2,4,6,8]]  # just want the Driver_Name, Price and Link and dexterity, flex and condition
    recommended['sim_score'] = scores            # add in the scores column
    recommended = recommended.sort_values(by = 'sim_score', ascending = False)
    recommended = recommended.drop_duplicates() # remove duplicates
    recommended = recommended.iloc[0:N, :] 
    
    return recommended 


In [65]:
'''
Function to run the recommender - prompt the user for input and use that to run recommender
'''
def run_recommender ():
    print("Please enter the variables you wish to use in your search. The top 5 recommendations will be provided")
    print("")
    print("Enter in the following order (you can leave some fields blank):")
    print("")
    print("Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords")
    print("")
    details = input(">")
    
    rec = recommend_driver(details.lower())
    print(rec)

## Below are some examples of recommendations based on different user inputs

### Initially I selected some fairly simple features such as brand name price and right handed. The results look good with all selections being the brand and handedness requested. The price feature doesn't seem to have had an influence, but that could because fuzzywuzzy isn't able to compare numbers. The flex that was selected as regular has only one match. The entries were made in capital letter just to ensure that worked.

In [66]:
run_recommender()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>TAYLORMADE $400 RIGHT REGULAR
           Driver_Name                                                                                     Link  Price Dexterity         Flex Condition  sim_score
22  taylormade  sim 2         https://www.golftraders.com.au/new-taylormade-sim-2-driver-10.5-graphite-hzrdus-r  779.0     right        stiff       new         92
76     taylormade  m6   https://www.golftraders.com.au/taylormade-m6-driver-9-graphite-stiff-flex-new-gri~46464  399.0     right        stiff      used         92
63     taylormade  m1         https://www.golftraders.com.au/taylormade-m1-driver-10.5-graphite-regular-flex-co  299.0     right      regular      used         92
31  taylormade  sim 2         https://www.golftraders.com.au/new-taylormad

## In the next case I tried a brand and left to get a left handed club. 3 of the 5 clubs were on the same brand enetered, but there were no left handed clubs selected. There are only a few left handed clubs in the dataset but it is disappointing that not one was selected. Also 3 of the 5 clubs we 'used' as was requested. This suggests some refinment of the recommendation engine is required.

In [34]:
run_recommender()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>callaway left $300 regular used


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
0,callaway mavrik,https://www.golftraders.com.au/new-callaway-mavrik-driver-10.5-graphite-with-shaf,549.0,right,options available,new,82
10,mizuno st190g,https://www.golftraders.com.au/mizuno-st190g-driver-9-graphite-stiff-flex-h5049,429.0,right,stiff,used,55
6,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,47
7,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,47
39,taylormade sim 2 max,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-10.5-ventus-5-regu,779.0,right,regular,new,43


## Again trying to get a left handed club but unfortunately using the word left alone does not select a left handed club. This suggests that a better method of entering the requirements is needed to filter through the results and give more accurate answers.

In [59]:
run_recommender()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>left handed


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
60,taylormade m1,https://www.golftraders.com.au/taylormade-m1-driver-10.5-graphite-regular-flex-co,299.0,right,regular,used,100
61,taylormade m1,https://www.golftraders.com.au/taylormade-m1-driver-10.5-graphite-regular-flex-co,299.0,right,regular,used,100
4,callaway great big bertha hawk eye,https://www.golftraders.com.au/callaway-great-big-bertha-hawk-eye-driver-9-graphi~45536,129.0,right,stiff,used,100
78,taylormade r1,https://www.golftraders.com.au/taylormade-r1-driver-8-12-graphite-regular-flex-co~46409,299.0,right,regular,used,100
6,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,100


## Now using a more general query such a 'best driver for player with high handicap'. This query is much more likely to use information from the reviews section as there were no specifications given. The prices were certainly cheaper for the top two recomendations. I would take that as an indication that the system is successful for a general query. From my own personal knowledge when you are starting golf and have a high handicap it is not recommended you have an expensive driver.

In [58]:
run_recommender()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>best driver for player with high handicap


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
4,callaway great big bertha hawk eye,https://www.golftraders.com.au/callaway-great-big-bertha-hawk-eye-driver-9-graphi~45536,129.0,right,stiff,used,92
6,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,92
85,titleist ts2,https://www.golftraders.com.au/titleist-ts2-driver-9.5-graphite-stiff-flex-cover~46179,469.0,left,stiff,used,84
14,new cleveland launcher hb turbo,https://www.golftraders.com.au/new-cleveland-launcher-hb-turbo-driver-10.5-graphi~45270,429.0,right,stiff (5554),new,84
28,taylormade sim 2,https://www.golftraders.com.au/new-taylormade-sim-2-driver-8-graphite-hzrdus-rdx,779.0,right,extra stiff,new,84


## Now to compare the recommeder without the review or verdict field. Thus relying on the specifications only.

In [51]:
'''
This function takes a series of user inputs taken from the run_recommender function and returns 
FuzzyWuzzy token_set_ratio between a list of drivers matching the inputs.
It needs to be this ratio as that ignores repated words.
'''

def recommend_driver_noreview(details):
    N = 5 #default number of results to return
    scores = []
    results = []
    length = df_combined.shape[0]
    
    for i in range(length):
            driver_details = df_combined.loc[i,'Driver_Name'] + str(df_combined.loc[i,'Driver_Name']) + str(df_combined.loc[i,'Price']) +\
                             str(df_combined.loc[i,'Loft'])  +  str(df_combined.loc[i,'Dexterity'])   + str(df_combined.loc[i,'Shaft']) +\
                             str(df_combined.loc[i,'Flex']) +str(df_combined.loc[i,'Condition'])+  str(df_combined.loc[i,'Condition'])
            driver_details = tokenize_and_lem_stem(driver_details) # stored values
            requested     = tokenize_and_lem_stem(str(details))    # user input values
            scores.append(fuzz.token_set_ratio(driver_details, requested)  ) # similarity measure 
    
    # which columns to extract
    
    recommended = df_combined.iloc[:,[0,1,2,4,6,8]]  # just want the Driver_Name, Price and Link and dexterity, flex and condition
    recommended['sim_score'] = scores            # add in the scores column
    recommended = recommended.sort_values(by = 'sim_score', ascending = False)
    recommended = recommended.drop_duplicates() # remove duplicates
    recommended = recommended.iloc[0:N, :] 
    
    return recommended 


In [60]:
'''
Function to run the recommender - prompt the user for input and use that to run recommender
'''
def run_recommender_noreview ():
    print("Please enter the variables you wish to use in your search. The top 5 recommendations will be provided")
    print("")
    print("Enter in the following order (you can leave some fields blank):")
    print("")
    print("Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords")
    print("")
    details = input(">")
    
    rec = recommend_driver_noreview(details.lower())
    display(rec)

## Running the recommender without the review/verdict and using the same queries as above. The results are more closeley aligned to the user input. 

In [53]:
run_recommender_noreview()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>TAYLORMADE $400 RIGHT REGULAR


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
61,taylormade m1,https://www.golftraders.com.au/taylormade-m1-driver-10.5-graphite-regular-flex-co,299.0,right,regular,used,57
2,taylormade sim 2,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-9-graphite-kuro-ka,779.0,right,stiff,new,51
38,taylormade sim 2 max,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-10.5-kuro-kage-reg,779.0,right,regular,new,50
41,taylormade sim 2 max,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-12-ventus-5-senior,779.0,right,seniors,new,50
40,taylormade sim 2 max,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-10.5-ventus-5-regu,779.0,right,regular,new,50


In [49]:
run_recommender_noreview ()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>callaway left $300 regular used


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
7,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,49
6,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,49
0,callaway mavrik,https://www.golftraders.com.au/new-callaway-mavrik-driver-10.5-graphite-with-shaf,549.0,right,options available,new,43
5,callaway great big bertha hawk eye,https://www.golftraders.com.au/callaway-great-big-bertha-hawk-eye-driver-9-graphi~45536,129.0,right,stiff,used,43
4,callaway great big bertha hawk eye,https://www.golftraders.com.au/callaway-great-big-bertha-hawk-eye-driver-9-graphi~45536,129.0,right,stiff,used,43


## In this case if we try again for 'left handed' the top 3 selctions are actually left handed. This is due to the fuzzy matching now only  having a few words to compare and so the word left has a much bigger influence.

In [62]:
run_recommender_noreview ()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>left handed


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
56,ping g25,https://www.golftraders.com.au/ping-g25-driver-10.5-graphite-regular-flex-left-ha,229.0,left,regular,used,22
85,titleist ts2,https://www.golftraders.com.au/titleist-ts2-driver-9.5-graphite-stiff-flex-cover~46179,469.0,left,stiff,used,18
83,taylormade tour burner,https://www.golftraders.com.au/taylormade-tour-burner-driver-10.5-graphite-stiff,169.0,left,stiff,used,18
81,taylormade r1,https://www.golftraders.com.au/taylormade-r1-driver-8-12-graphite-regular-flex-co~46409,299.0,right,regular,used,18
6,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,18


In [61]:
run_recommender_noreview ()

Please enter the variables you wish to use in your search. The top 5 recommendations will be provided

Enter in the following order (you can leave some fields blank):

Brand, Handedness, Price, Loft, Shaft Flex, Grip, Condition, Any Keywords

>best driver for player with high handicap


,Driver_Name,Link,Price,Dexterity,Flex,Condition,sim_score
43,taylormade sim 2,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-9-graphite-ventus,779.0,right,stiff,new,34
7,callaway great big bertha war bird,https://www.golftraders.com.au/callaway-great-big-bertha-war-bird-driver-11-graph~46093,99.0,right,regular,used,34
78,taylormade r1,https://www.golftraders.com.au/taylormade-r1-driver-8-12-graphite-regular-flex-co~46409,299.0,right,regular,used,34
61,taylormade m1,https://www.golftraders.com.au/taylormade-m1-driver-10.5-graphite-regular-flex-co,299.0,right,regular,used,33
3,taylormade sim 2,https://www.golftraders.com.au/new-taylormade-sim-2-max-driver-9-graphite-kuro-ka,779.0,right,stiff,new,33
